# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1024 07:34:34.188473    6769 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1024 07:34:49.921603    6801 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Mon Oct 24 07:34:52 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [128]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/distributed/mnist_mnistcnn/epoch_25/experiment_10e-1_25.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [129]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1024 12:18:09.965758   29570 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1024 12:18:16.350660   29586 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Mon Oct 24 12:18:18 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [130]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1024 12:18:31.198415   29624 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-24-2022 10:18:24 root         INFO     Loading file config/configuration.fltk.json
10-24-2022 10:18:24 root         INFO     Starting in cluster mode.
10-24-2022 10:18:24 root         INFO     Starting with experiment replication: 0 with seed: 42
No argument path is provided.
10-24-2022 10:18:24 root         INFO     Starting as Orchestrator
10-24-2022 10:18:24 root         INFO     Starting Orchestrator, initializing resources....
10-24-2022 10:18:24 root         INFO     Loading in cluster configuration file
10-24-2022 10:18:24 root         INFO     Pointing configuration to in cluster configuration.
10-24-2022 10:18:24 root         INFO     Starting cluster manager
10-24-2022 10:18:24 ClusterManager INFO     Spinning up cluster manag

In [182]:
CURRENT_POD_ID="cef51bef-8e92-447c-bde2-a71385e36bc6"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1024 13:29:09.402771    3310 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-24-2022 10:54:19 root         INFO     Loading file config/configuration.fltk.json
10-24-2022 10:54:19 root         INFO     Starting in client mode
10-24-2022 10:54:19 root         INFO     Starting with host=trainjob-cef51bef-8e92-447c-bde2-a71385e36bc6-master-0 and port=23456
10-24-2022 10:54:19 root         INFO     Initializing backend for training process: gloo
10-24-2022 11:16:10 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-24-2022 11:16:10 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-24-2022 11:16:10 root         INFO     Starting Creating client with 0
10-24-2022 11:16:10 Client-0-c

/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-24-2022 11:23:27 Client-0-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,     0] loss: 0.046
10-24-2022 11:23:39 Client-0-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,    50] loss: 2.333
10-24-2022 11:23:51 Client-0-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,   100] loss: 2.329
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-24-2022 11:23:58 Client-0-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [16,     0] loss: 0.046
10-24-2022 11:24:10 Client-0-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [16,    50] loss: 2.332
10-24-2022 11:24:22 Client-0-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [16,   100] loss: 2.328
/opt/feder

       [ 44,   0,   9,   3,   5,  33,  11,   0,  41, 124],
       [ 25,   1,   8,   5,   5,  47,  15,   0,  43, 106],
       [ 30,   0,   2,   2,   4,  34,  18,   0,  18, 122],
       [ 25,   0,   6,   3,   2,  33,  11,   0,  32, 109],
       [ 36,   1,   1,   4,   6,  33,  37,   2,  19, 105],
       [ 21,   2,  11,   4,   9,  32,  14,   0,  28, 115],
       [ 38,   0,   2,  10,   9,  32,  21,   1,  27,  97]])), EpochData(epoch_id=4, num_epochs=26, duration_train=122374, duration_test=2790, loss_train=2.332239575386047, accuracy=9.36, loss=46.71188020706177, class_precision=array([0.08035714, 0.22222222, 0.13432836, 0.11764706, 0.15909091,
       0.0819209 , 0.07407407, 0.        , 0.12280702, 0.08900524]), class_recall=array([0.1       , 0.00746269, 0.03345725, 0.01481481, 0.02745098,
       0.12608696, 0.05429864, 0.        , 0.1779661 , 0.43037975]), confusion_mat=array([[ 27,   0,   8,   3,   2,  46,  10,   0,  38, 136],
       [ 46,   2,   7,   2,   5,  38,  18,   1,  39, 110],
  

       0.13913043, 0.0361991 , 0.        , 0.13559322, 0.40084388]), confusion_mat=array([[ 31,   1,  13,   8,   4,  44,   5,   0,  36, 128],
       [ 61,   1,   5,   4,   4,  42,  14,   0,  37, 100],
       [ 30,   0,   6,   4,   8,  31,  16,   1,  41, 132],
       [ 40,   1,   8,   2,   5,  42,  14,   0,  47, 111],
       [ 22,   2,   5,   6,   8,  34,  13,   2,  44, 119],
       [ 27,   0,   4,   5,   2,  32,  21,   0,  23, 116],
       [ 30,   0,  13,   4,   3,  40,   8,   0,  27,  96],
       [ 41,   0,   3,   1,   3,  35,  24,   0,  25, 112],
       [ 18,   1,   7,   2,   4,  34,  25,   1,  32, 112],
       [ 36,   2,   2,   7,   7,  31,  23,   0,  34,  95]])), EpochData(epoch_id=12, num_epochs=26, duration_train=370979, duration_test=2876, loss_train=2.3258898973464968, accuracy=8.96, loss=46.68481111526489, class_precision=array([0.08722741, 0.        , 0.07407407, 0.09803922, 0.15686275,
       0.09444444, 0.06547619, 0.        , 0.10479042, 0.08631212]), class_recall=array([0

       0.09166667, 0.03571429, 0.        , 0.08405797, 0.09341637]), class_recall=array([0.12962963, 0.        , 0.02973978, 0.0037037 , 0.03529412,
       0.14347826, 0.02714932, 0.        , 0.12288136, 0.44303797]), confusion_mat=array([[ 35,   1,  10,   7,   6,  46,  12,   0,  47, 106],
       [ 68,   0,   9,   1,   4,  33,  11,   0,  38, 104],
       [ 27,   1,   8,   5,   3,  42,  20,   1,  46, 116],
       [ 41,   1,   8,   1,   7,  39,  16,   0,  36, 121],
       [ 23,   2,   6,   2,   9,  33,   7,   1,  47, 125],
       [ 35,   0,   1,   4,   6,  33,  27,   0,  26,  98],
       [ 30,   2,   4,   3,   3,  33,   6,   0,  32, 108],
       [ 27,   0,   2,   0,   9,  39,  24,   0,  26, 117],
       [ 17,   2,   8,   2,   6,  29,  19,   0,  29, 124],
       [ 42,   0,   1,   6,   6,  33,  26,   0,  18, 105]])), EpochData(epoch_id=20, num_epochs=26, duration_train=619454, duration_test=2812, loss_train=2.3302979326248168, accuracy=9.36, loss=46.582793951034546, class_precision=array([

In [183]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1024 13:29:16.022839    3329 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-24-2022 10:54:22 root         INFO     Loading file config/configuration.fltk.json
10-24-2022 10:54:22 root         INFO     Starting in client mode
10-24-2022 10:54:22 root         INFO     Starting with host=trainjob-cef51bef-8e92-447c-bde2-a71385e36bc6-master-0 and port=23456
10-24-2022 10:54:22 root         INFO     Initializing backend for training process: gloo
10-24-2022 11:16:10 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-24-2022 11:16:10 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-24-2022 11:16:10 root         INFO

10-24-2022 11:23:20 Client-1-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [14,   100] loss: 2.331
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-24-2022 11:23:27 Client-1-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,     0] loss: 0.046
10-24-2022 11:23:39 Client-1-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,    50] loss: 2.337
10-24-2022 11:23:51 Client-1-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,   100] loss: 2.327
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-24-2022 11:23:58 Client-1-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [16,     0] loss: 0.047
10-24-2022 11:24:10 Client-1-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [16,    50] loss: 2.338
10-24-2022

       0.13432836, 0.02755906, 0.        , 0.12288136, 0.44094488]), confusion_mat=array([[ 25,   0,   6,   3,   4,  36,  13,   0,  43, 108],
       [ 57,   1,   5,   3,   2,  35,  21,   0,  34, 111],
       [ 27,   1,   5,   3,   4,  33,  15,   0,  29, 122],
       [ 41,   0,   6,   3,   1,  42,  13,   1,  53,  97],
       [ 28,   0,  16,   6,   3,  33,  10,   0,  42,  92],
       [ 42,   1,   3,   1,   3,  36,  24,   0,  30, 128],
       [ 31,   0,  10,   4,   5,  34,   7,   1,  41, 121],
       [ 44,   1,   2,   1,   8,  32,  33,   0,  24, 110],
       [ 21,   0,   9,   4,   6,  30,  16,   0,  29, 121],
       [ 41,   0,   2,  11,   7,  39,  17,   0,  25, 112]])), EpochData(epoch_id=4, num_epochs=26, duration_train=122385, duration_test=2715, loss_train=2.3348521423339843, accuracy=9.72, loss=46.68444585800171, class_precision=array([0.1       , 0.        , 0.09615385, 0.04      , 0.13636364,
       0.11466667, 0.08387097, 0.        , 0.11258278, 0.09044801]), class_recall=array([0.

       0.12250712, 0.07486631, 0.        , 0.10857143, 0.09165154]), class_recall=array([0.10084034, 0.00371747, 0.0251046 , 0.0077821 , 0.04347826,
       0.16044776, 0.05511811, 0.        , 0.16101695, 0.3976378 ]), confusion_mat=array([[ 24,   2,   7,   2,   8,  40,   9,   1,  36, 109],
       [ 54,   1,   9,   1,   4,  42,  20,   0,  46,  92],
       [ 25,   2,   6,   6,   5,  35,  11,   0,  47, 102],
       [ 38,   0,   9,   2,   5,  33,  13,   1,  34, 122],
       [ 20,   1,  10,   2,  10,  23,  17,   0,  41, 106],
       [ 47,   1,   2,   3,   4,  43,  21,   0,  25, 122],
       [ 29,   1,   8,   4,   7,  36,  14,   0,  33, 122],
       [ 45,   0,   1,   4,  11,  28,  32,   0,  24, 110],
       [ 16,   2,   6,   2,   6,  35,  15,   0,  38, 116],
       [ 43,   1,   2,   3,   7,  36,  35,   0,  26, 101]])), EpochData(epoch_id=12, num_epochs=26, duration_train=370990, duration_test=2802, loss_train=2.325100374221802, accuracy=9.96, loss=46.72688674926758, class_precision=array([0.

       0.08635097, 0.04597701, 0.        , 0.08683473, 0.09645132]), class_recall=array([0.09663866, 0.        , 0.0334728 , 0.01167315, 0.03913043,
       0.11567164, 0.03149606, 0.        , 0.13135593, 0.41732283]), confusion_mat=array([[ 23,   1,   7,   1,   6,  36,  13,   1,  38, 112],
       [ 54,   0,   9,   0,   7,  39,  17,   0,  46,  97],
       [ 25,   0,   8,   3,   6,  32,  10,   3,  39, 113],
       [ 42,   2,   7,   3,   4,  37,  15,   0,  46, 101],
       [ 30,   1,  11,   4,   9,  30,  11,   1,  38,  95],
       [ 33,   1,   1,   3,   2,  31,  22,   0,  30, 145],
       [ 30,   0,   8,   1,   5,  45,   8,   1,  40, 116],
       [ 37,   1,   5,   3,   8,  42,  27,   0,  22, 110],
       [ 21,   1,   9,   5,   6,  37,  20,   2,  31, 104],
       [ 40,   0,   2,  11,   6,  30,  31,   1,  27, 106]])), EpochData(epoch_id=20, num_epochs=26, duration_train=619465, duration_test=2724, loss_train=2.332163052558899, accuracy=9.4, loss=46.642306327819824, class_precision=array([0.

In [184]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1024 13:29:21.535010    3350 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-24-2022 11:16:10 root         INFO     Loading file config/configuration.fltk.json
10-24-2022 11:16:10 root         INFO     Starting in client mode
10-24-2022 11:16:10 root         INFO     Starting with host=trainjob-cef51bef-8e92-447c-bde2-a71385e36bc6-master-0 and port=23456
10-24-2022 11:16:10 root         INFO     Initializing backend for training process: gloo
10-24-2022 11:16:10 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-24-2022 11:16:10 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-24-2022 11:16:10 root         INFO

10-24-2022 11:23:20 Client-2-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [14,   100] loss: 2.328
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-24-2022 11:23:27 Client-2-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,     0] loss: 0.046
10-24-2022 11:23:39 Client-2-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,    50] loss: 2.333
10-24-2022 11:23:51 Client-2-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [15,   100] loss: 2.329
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-24-2022 11:23:58 Client-2-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [16,     0] loss: 0.046
10-24-2022 11:24:10 Client-2-cef51bef-8e92-447c-bde2-a71385e36bc6 INFO     [16,    50] loss: 2.332
10-24-2022

       [ 44,   0,   5,   2,   3,  43,   9,   0,  39,  85],
       [ 24,   1,   4,   0,   8,  34,  13,   0,  39, 111],
       [ 42,   0,   7,   4,   2,  40,   9,   1,  41, 106],
       [ 40,   1,  10,   8,   9,  28,  12,   0,  46, 107],
       [ 30,   0,   3,   1,   5,  43,  22,   1,  15, 116],
       [ 27,   3,  10,   5,   2,  33,  14,   0,  30, 145],
       [ 44,   1,   6,   3,   8,  36,  20,   0,  19, 117],
       [ 26,   2,   6,   4,   8,  32,  22,   1,  27, 143],
       [ 40,   1,   3,   7,   5,  33,  27,   0,  32, 101]])), EpochData(epoch_id=4, num_epochs=26, duration_train=122386, duration_test=2791, loss_train=2.332239575386047, accuracy=9.2, loss=46.71070384979248, class_precision=array([0.07232704, 0.        , 0.10447761, 0.07894737, 0.1509434 ,
       0.0862069 , 0.10240964, 0.        , 0.12256267, 0.08596491]), class_recall=array([0.0942623 , 0.        , 0.02991453, 0.01190476, 0.03065134,
       0.12711864, 0.06319703, 0.        , 0.16236162, 0.3935743 ]), confusion_mat=arr

       0.1440678 , 0.05204461, 0.        , 0.16236162, 0.37349398]), confusion_mat=array([[ 39,   1,  12,   5,   4,  30,  10,   1,  33, 109],
       [ 51,   2,   6,   2,   3,  36,  12,   1,  30,  87],
       [ 25,   1,   9,   6,   4,  38,  13,   0,  33, 105],
       [ 43,   2,   0,   1,   2,  36,  15,   0,  41, 112],
       [ 29,   1,   6,   5,   5,  37,  12,   1,  44, 121],
       [ 32,   0,   3,   4,   3,  34,  14,   1,  16, 129],
       [ 30,   1,   7,   3,   8,  42,  14,   0,  39, 125],
       [ 50,   0,   4,   2,   5,  28,  27,   0,  17, 121],
       [ 23,   1,   8,   5,   2,  49,  22,   0,  44, 117],
       [ 37,   2,   4,   8,   7,  38,  28,   0,  32,  93]])), EpochData(epoch_id=12, num_epochs=26, duration_train=370991, duration_test=2577, loss_train=2.3258898973464968, accuracy=10.2, loss=46.545185804367065, class_precision=array([0.07643312, 0.5       , 0.1147541 , 0.07142857, 0.19402985,
       0.09333333, 0.07692308, 0.        , 0.12962963, 0.10162254]), class_recall=array([

       0.09887006, 0.08333333, 0.        , 0.11337209, 0.09652174]), class_recall=array([0.12704918, 0.        , 0.01709402, 0.00396825, 0.03448276,
       0.14830508, 0.04832714, 0.        , 0.14391144, 0.44578313]), confusion_mat=array([[ 31,   0,   6,   2,   2,  41,   9,   0,  48, 105],
       [ 46,   0,   9,   3,   2,  32,  18,   0,  34,  86],
       [ 24,   0,   4,   6,   5,  29,  13,   0,  40, 113],
       [ 58,   2,   7,   1,   6,  45,  13,   0,  25,  95],
       [ 15,   1,   7,   3,   9,  32,  15,   0,  48, 131],
       [ 37,   1,   1,   7,   2,  35,  10,   0,  18, 125],
       [ 20,   2,   6,   4,   8,  34,  13,   2,  44, 136],
       [ 41,   0,   2,   6,   6,  38,  19,   0,  25, 117],
       [ 22,   2,   5,   6,   9,  35,  20,   2,  39, 131],
       [ 34,   0,   4,  10,   8,  33,  26,   0,  23, 111]])), EpochData(epoch_id=20, num_epochs=26, duration_train=619466, duration_test=2804, loss_train=2.3302979326248168, accuracy=9.76, loss=46.41840624809265, class_precision=array([0

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [25]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1023 19:03:06.867426   26766 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1023 19:03:07.651839   26780 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [26]:
helm uninstall -n test experiment-orchestrator

W1023 19:03:18.736456   26811 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [27]:
helm uninstall extractor -n test

W1023 19:03:46.312453   26913 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [28]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1023 19:05:56.705875   27066 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-3ca1281a-a577-4be9-a4d0-512769d1e1d5" deleted
pytorchjob.kubeflow.org "trainjob-8c75fbb2-7172-46a3-b9f2-64fd122ca38d" deleted
pytorchjob.kubeflow.org "trainjob-9a7ddfd7-6eee-4694-acfb-f923de6bee62" deleted
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
